# Задача

В рамках данной задачи предстоит научиться получать наилучшее приближение $\sigma\in\mathbb{C}^n$ истинного квантового состояния $\rho\in\mathbb{C}^n$ по некоторой обучающей выборке $X_{train}=\{E_m,y_m\}_{m=0}^{N}$, где $E_m$ - некоторая матрица (называемая оператор, проектор), $y_m$ - некоторое наблюдение (проекция истинного состояния $\rho$ на этот оператор). 

В качестве оптимизируемого функционала предлагается минимизировать следующую функцию:
$$min_\sigma[f(\sigma)=\Sigma_m(Tr(E_m\sigma)-Tr(E_m\rho))^2=\Sigma_m(Tr(E_m\sigma)-y_m)^2]$$
$$\sigma\geq0,Tr(\sigma)=1$$
гдe $\sigma\geq0$ означает положительную полуопределённость обучаемой матрицы $\sigma$ 

# Решение
Поскольку матрица $\sigma$ - положительно полуопределённая, сделаем разложение Холецкого: $\sigma=M^*M$. 

Тогда 
$$Tr(\sigma)=Tr(M^*M)=\Sigma_{a_m\in M} a_m^*a_m=\Sigma_{a_m\in M} |a_m|^2$$

Поскольку для любой положительно полуопределённой матрицы существует разложение Холецкого, и, наоборот, для любой матрицы $M\in\mathbb{C}^n$ матрица $M^*M$ является положительно полуопределённой, мы можем переписать оптимизационную задачу в виде

$$min_M[f(M)=\Sigma_m(Tr(E_mM^*M)-y_m)^2]$$
$$Tr(M^*M)=\Sigma_{a_m\in M}|a_m|^2=1$$

Эту задачу можно было бы решать обычным методом множителей Лагранжа, но можно просто перенормировать $M$ на $\sqrt{Tr(M^*M)}$. Тогда задача сведётся к задаче безусловной оптимизации

$$min_M[f(M)=\Sigma_m(Tr(E_m\frac{M^*M}{Tr(M^*M)})-y_m)^2=\Sigma_m(Tr(E_m\frac{M^*M}{\Sigma_{a_m\in M}|a_m|^2})-y_m)^2]$$

# Код

## Генерация датасета

In [1]:
import numpy as np
import simulator
rho = np.array([[1., 0.], [0., 0.]]) # target state
dim = rho.shape[0]
train_size = 2
train_X, train_y = simulator.generate_dataset(rho, 2)

## Обучение модели

In [2]:
import torch
import sys
sys.path.append('pytorch-complex-tensor/')
from pytorch_complex_tensor import ComplexTensor
M = ComplexTensor(simulator.randomMixedState(rho.shape[0]))
M.requires_grad = True

In [33]:
from torch.optim import SGD
opt = SGD([M], lr=1e-1)
epoches = 20
def trace(tensor):
    return sum([tensor[i:i+1,i:i+1] for i in range(tensor.shape[0])])

for epoch in range(epoches):
    sigma = M.t(conjugate=True).mm(M)
    norm = trace(sigma).real.sum()

    loss = 0
    for E_m,y_m in zip(train_X, train_y.astype('float64')):
        E_m = ComplexTensor(E_m)
        loss += ((trace(E_m.mm(sigma)/norm)-y_m)**2).sum()
    loss /= train_size
    loss.backward()
    opt.step()
    opt.zero_grad()
    print(f'Epoch: {epoch}, Loss: {loss.detach().cpu().numpy()}')

Epoch: 0, Loss: 1.6041244268417358
Epoch: 1, Loss: 1.6036443710327148
Epoch: 2, Loss: 1.6032569408416748
Epoch: 3, Loss: 1.6029387712478638
Epoch: 4, Loss: 1.6026734113693237
Epoch: 5, Loss: 1.602449893951416
Epoch: 6, Loss: 1.6022610664367676
Epoch: 7, Loss: 1.6020973920822144
Epoch: 8, Loss: 1.6019563674926758
Epoch: 9, Loss: 1.601832628250122
Epoch: 10, Loss: 1.6017251014709473
Epoch: 11, Loss: 1.6016281843185425
Epoch: 12, Loss: 1.6015437841415405
Epoch: 13, Loss: 1.6014670133590698
Epoch: 14, Loss: 1.6013984680175781
Epoch: 15, Loss: 1.6013376712799072
Epoch: 16, Loss: 1.601282000541687
Epoch: 17, Loss: 1.6012306213378906
Epoch: 18, Loss: 1.6011852025985718
Epoch: 19, Loss: 1.6011428833007812


In [34]:
sigma = sigma/norm
sigma = sigma.detach().numpy()[:dim] + 1j*sigma.detach().numpy()[dim:]
sigma

array([[0.91788   +0.j        , 0.16271587+0.22112666j],
       [0.16271587-0.22112666j, 0.08211999+0.j        ]], dtype=complex64)

In [35]:
simulator.isDensityMatrix(sigma)

True